In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np




D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:461: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [2]:
CIFAR_DIR = 'E:/jupyter/cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [13]:
def load_data(filename):
    #read data form data file
    with open(filename,'rb') as f:
        data = pickle.load(f,encoding='bytes')
#         print(type(data))
#         print(data.keys())
        
#         print(type(data[b'labels']))
        return data[b'data'],data[b'labels']

#tesorflow.Dataset
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename)
            for item,label in zip(data,labels):
                if label in [0,1]:
                    all_data.append(item)
                    all_labels.append(label)
        self._data = np.vstack(all_data)
        self._data = self._data/127.5 - 1  #归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples) #直接调用numoy的混排函数
        self._data = self._data[p]
        self._labels = self._labels[p]
    def next_batch(self,batch_size):
        #return batch_size examples as a batch
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator>self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data= self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels

train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d'%i) for i in range(1,6)]  
test_filenames = [os.path.join(CIFAR_DIR,'test_batch')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,False)

(10000, 3072)
(10000,)
(2000, 3072)
(2000,)


In [4]:
x = tf.placeholder(tf.float32,[None,3072])
#[None]
y = tf.placeholder(tf.int64,[None])

#(3072,1)
w = tf.get_variable('w',[x.get_shape()[-1],1],
                    initializer=tf.random_normal_initializer(0,1)
                   )
#(1,)
b = tf.get_variable('b',[1],initializer=tf.constant_initializer(0.0))

#[None,3072]*[3072,1] = [None,1]
y_ = tf.matmul(x,w)+b

#[None,1]
p_y_1 = tf.nn.sigmoid(y_)
#[None,1]
y_reshape = tf.reshape(y,(-1,1))
#in order to become the same type -float32
y_reshape_float = tf.cast(y_reshape,tf.float32)

loss = tf.reduce_mean(tf.square(y_reshape_float-p_y_1))

#bool
predict = p_y_1>0.5
correct_prediction = tf.equal(tf.cast(predict,tf.int64),y_reshape)
acc = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdadeltaOptimizer(1e-3).minimize(loss)


In [14]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 50000

test_steps = 100
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        loss_val,acc_val,_ = sess.run(
            [loss,acc,train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels
            }
        )
        if i%500 ==0:
            print('[Train] Steps: %d, loss: %4.5f, acc: %4.5f'%(i,loss_val,acc_val))
        
        if (i+1)%5000 == 0:
            test_data = CifarData(test_filenames,False)
            for j in range(test_steps):
                test_batch_data,test_batch_labels = test_data.next_batch(batch_size)
                all_test_acc_val = []
                test_acc_val = sess.run(
                    [acc],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    }
                )
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1,test_acc))
        

[Train] Steps: 0, loss: 0.63886, acc: 0.35000
[Train] Steps: 500, loss: 0.54977, acc: 0.45000
[Train] Steps: 1000, loss: 0.46767, acc: 0.50000
[Train] Steps: 1500, loss: 0.69986, acc: 0.30000
[Train] Steps: 2000, loss: 0.54753, acc: 0.45000
[Train] Steps: 2500, loss: 0.57072, acc: 0.40000
[Train] Steps: 3000, loss: 0.52742, acc: 0.45000
[Train] Steps: 3500, loss: 0.50671, acc: 0.50000
[Train] Steps: 4000, loss: 0.74374, acc: 0.25000
[Train] Steps: 4500, loss: 0.52418, acc: 0.45000
(2000, 3072)
(2000,)
[Test] Step: 5000, acc: 0.30000
[Train] Steps: 5000, loss: 0.46161, acc: 0.55000
[Train] Steps: 5500, loss: 0.64850, acc: 0.35000
[Train] Steps: 6000, loss: 0.54883, acc: 0.45000
[Train] Steps: 6500, loss: 0.73450, acc: 0.25000
[Train] Steps: 7000, loss: 0.46632, acc: 0.50000
[Train] Steps: 7500, loss: 0.55050, acc: 0.45000
[Train] Steps: 8000, loss: 0.68185, acc: 0.30000
[Train] Steps: 8500, loss: 0.50169, acc: 0.50000
[Train] Steps: 9000, loss: 0.51919, acc: 0.45000
[Train] Steps: 9500,